## Environment set-up

In [1]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


Import Libraries

In [2]:
import subprocess
import os
import sys
import platform
import shutil
import json

---

Define Function to Create and Setup Virtual Environment

In [3]:
def create_and_setup_virtual_environment():
    venv_dir = 'myenv'
    python_executable = sys.executable
    
    # Check if the virtual environment directory already exists
    if os.path.exists(venv_dir):
        # Remove the existing virtual environment directory
        print(f"Removing existing virtual environment directory '{venv_dir}'.")
        shutil.rmtree(venv_dir)
    
    # Create a new virtual environment
    subprocess.run([python_executable, '-m', 'venv', venv_dir], check=True)
    print(f"Created a new virtual environment in '{venv_dir}'.")

    # Determine the path to the Python executable in the virtual environment
    if platform.system() == 'Windows':
        venv_python = os.path.join(venv_dir, 'Scripts', 'python')
    else:
        venv_python = os.path.join(venv_dir, 'bin', 'python')
    
    # Install Gradio in the new virtual environment
    subprocess.run([venv_python, '-m', 'pip', 'install', 'gradio'], check=True)
    print("Installed Gradio in the virtual environment.")

Create the Virtual Environment and Install Gradio

In [4]:
create_and_setup_virtual_environment()

Removing existing virtual environment directory 'myenv'.
Created a new virtual environment in 'myenv'.
Installed Gradio in the virtual environment.


Define Function to Clone or Update Repository

In [5]:
def clone_or_update_repository():
    repo_url = 'https://github.com/ProjectPhysX/FluidX3D'
    clone_dir = 'FluidX3D'
    
    if os.path.exists(clone_dir) and os.path.isdir(os.path.join(clone_dir, '.git')):
        # If it's a Git repository, pull the latest changes
        print(f"Updating the repository in '{clone_dir}'.")
        subprocess.run(['git', '-C', clone_dir, 'pull'], check=True)
    elif not os.path.exists(clone_dir):
        # If the directory does not exist, clone the repository
        subprocess.run(['git', 'clone', repo_url], check=True)
    else:
        # If the directory exists but is not a Git repository
        print(f"The directory '{clone_dir}' already exists and is not a git repository. Skipping clone.")


## Build FluidX3D

Find and compile solution

In [6]:
def find_msbuild():
    # Path to vswhere.exe - adjust if necessary
    vswhere_path = r"C:\Program Files (x86)\Microsoft Visual Studio\Installer\vswhere.exe"
    # Use vswhere to find the installation path for MSBuild
    try:
        result = subprocess.run([vswhere_path, "-latest", "-requires", "Microsoft.Component.MSBuild", "-find", "MSBuild\\**\\Bin\\MSBuild.exe", "-format", "json"], capture_output=True, text=True, check=True)
        installations = json.loads(result.stdout)
        if installations:
            return installations[0]  # Returns the path to the latest MSBuild executable
    except subprocess.CalledProcessError as e:
        print(f"Error finding MSBuild with vswhere: {e}")
    return None

def compile_solution(msbuild_path, solution_path):
    try:
        result = subprocess.run([msbuild_path, solution_path, "/p:Configuration=Release"], capture_output=True, text=True, check=False)
        if result.returncode != 0:
            print("Failed to compile the solution:")
            print(result.stdout)
            print(result.stderr)
        else:
            print("Solution compiled successfully.")
            print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

Run solution

In [7]:
def run_application():
    executable_path = os.path.join(os.getcwd(), "FluidX3D", "bin", "FluidX3D")
    # Command to open a new console window
    command = f'start cmd /K "{executable_path}"'
    try:
        subprocess.run(command, shell=True)
    except Exception as e:
        print(f"Failed to run the application: {e}")

In [8]:
def compile_and_play_simulation():
    msbuild_path = find_msbuild()
    solution_path = os.path.join(os.getcwd(), "FluidX3D", "FluidX3D.sln")
    compile_solution(msbuild_path, solution_path)
    run_application()

## Expose FluidX3D as Gradio interface

In [19]:
import gradio as gr

def read_current_states(definitions_path):
    states = {}
    with open(definitions_path, 'r') as file:
        lines = file.readlines()
    for line in lines:
        if line.strip().startswith('#define') or line.strip().startswith('//#define'):
            active = line.strip().startswith('#define')
            parts = line.strip().split()
            if len(parts) > 1:
                define_part = parts[1]
                states[define_part] = active
    return states

def update_definitions(definitions_path, **states):
    with open(definitions_path, 'r') as file:
        lines = file.readlines()
    new_lines = []
    for line in lines:
        modified = False
        for define, active in states.items():
            if line.strip().startswith(f'#define {define}') or line.strip().startswith(f'//#define {define}'):
                line = f"{'#' if not active else ''}define {define}\n"
                modified = True
                break
        if not modified:
            new_lines.append(line)
    with open(definitions_path, 'w') as file:
        file.writelines(new_lines)
    return "Updated definitions successfully!"

definitions_path = os.path.join(os.getcwd(), "FluidX3D", "src", "defines.hpp")
current_states = read_current_states(definitions_path)

def create_interface():
    inputs = []
    for define, active in current_states.items():
        checkbox = gr.Checkbox(label=define, value=active)
        state_label = "Commented Out" if not active else "Active"
        inputs.append(gr.Row([checkbox, gr.Textbox(default=state_label, readonly=True, visible=False)]))
    
    iface = gr.Interface(
        fn=lambda **kwargs: update_definitions(definitions_path, **kwargs),
        inputs=inputs,
        outputs="text",
        title="Toggle .hpp Definitions",
        description="Toggle options to comment/uncomment definitions in the .hpp file. The state of each option is indicated next to the checkbox."
    )
    return iface

iface = create_interface()
iface.launch()


TypeError: Textbox.__init__() got an unexpected keyword argument 'default'